In [8]:
import pandas
import numpy
import datetime
import os
os.environ["CDF_LIB"] = "cdf36_4-dist/lib"
from spacepy import pycdf
import seaborn as sns
import aacgmv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib.dates import date2num, DateFormatter, MinuteLocator
from matplotlib.ticker import FuncFormatter, MaxNLocator
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
outBaseDir = "/home/bharat/Documents/code/scwsaps/data/mag/"
inpDate = datetime.datetime(2011,5,16)

In [ ]:
# read station data
# read_csv can't handle this!
stnCodeArr = []
stnNameArr = []
stnMlatArr = []
stnMlonArr = []
fStn = open(outBaseDir + "Station-Code-19700101.txt")
for nl, ll in enumerate(fStn.readlines()):
    if nl > 0:
        lineData = ll.split()
        stnCodeArr.append( lineData[0] )
        stnNameArr.append( lineData[3] )
        stnMlatArr.append( lineData[-5] )
        stnMlonArr.append( lineData[-4] )
# convert to dataframe
thmStnDF = pandas.DataFrame( {
    "station":stnCodeArr,
    "name":stnNameArr,
    "mlat":stnMlatArr,
    "mlon":stnMlonArr
} )
thmStnDF.head()

,mlat,mlon,name,station
0,65.49,101.09,"Abisko,",ABK
1,68.75,52.69,"Tasiilaq,",AMK
2,65.67,136.68,"Amderma,",AMD
3,48.79,331.29,"Americus,",AMER
4,66.6,99.44,"Andenes,",AND


In [13]:
# read the actual magnetometer data
t = "/home/bharat/Documents/code/scwsaps/data/mag/thg_20110516/thg_l2_mag_amer_20110516_v01.cdf"
currCdf = pycdf.CDF(t)
print currCdf

range_epoch: CDF_EPOCH [2]
thg_mag_amer: CDF_REAL4 [172800, 3]
thg_mag_amer_compno: CDF_INT4 [3] NRV
thg_mag_amer_epoch: CDF_EPOCH [0]
thg_mag_amer_epoch0: CDF_EPOCH [] NRV
thg_mag_amer_labl: CDF_CHAR*21 [3] NRV
thg_mag_amer_time: CDF_REAL8 [172800]
thg_mag_amer_unit: CDF_CHAR*2 [3] NRV


In [ ]:
for ii in currCdf["thg_mag_amer"]:
    print ii